# Colab Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
"""
Change directory to where this file is located
"""
%cd 'COPY&PASTE FILE DIRECTORY HERE'

# Import Modules

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import transforms, datasets

from tqdm.auto import tqdm

# Utils

In [3]:
def train(model, train_loader, optimizer, criterion, DEVICE):
    """
    Trains the model with training data.

    Do NOT modify this function.
    """
    model.train()
    tqdm_bar = tqdm(train_loader)
    for batch_idx, (image, label) in enumerate(tqdm_bar):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        tqdm_bar.set_description("Epoch {} - train loss: {:.6f}".format(epoch, loss.item()))


def evaluate(model, test_loader, criterion, DEVICE):
    """
    Evaluates the trained model with test data.

    Do NOT modify this function.
    """
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for image, label in tqdm(test_loader):
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim=True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

# ViT Model

In [ ]:
###
# Always check tensor shapes!
# Printing shapes can be the fastest way to track the error and fix it.
# Using dropout is optional. You don't have to use pre-declared dropouts.
###

In [4]:
class Patchification(nn.Module):
  """
  Question (a)
  Process the batch of images to non-overlapping patches using convolution layer.
  You are only allowed to use torch.nn.Con2d

  - Input shape: [batch, channel, height, width]
  - Return: [batch, number_of_patches, embedding_dimension]
  """
  def __init__(self, in_channels, patch_size, embedding_dim):
    super().__init__()
    """
    Hint: embedding_dim should be the out_channel of convolution.
    """
    ##### YOUR CODE #####
    self.conv = nn.Conv2d(in_channels, embedding_dim, kernel_size=patch_size, stride=patch_size)
    #####################

  def forward(self, x):
    ##### YOUR CODE #####
    x = self.conv(x)
    x = x.flatten(2)
    x = x.transpose(1, 2)
    #####################
    return x

In [5]:
"""
Hint cell.
You can briefly check if the function is implemented correctly by printing the shape of the output before jumping into the training with GPU.
Example code is written in the below.
"""
batch, channel, height, width = 5, 3, 32, 32
patch_size = (4, 4)
embedding_dim = 128

test_img = torch.ones(batch, channel, height, width)

patchification = Patchification(channel, patch_size, embedding_dim)
after_patchification = patchification(test_img)

print(after_patchification.shape) # Is the shape [batch, number_of_patches, dim] ?

torch.Size([5, 64, 128])


In [6]:
class MLP(nn.Module):
  """
  Feed-forward layer

  - Input shape: [batch, number_of_patches, embedding_dimension]
  - Return: [batch, number_of_patches, embedding_dimension]

  Do NOT modify.
  """
  def __init__(self, dim, hidden_dim, dropout = 0.):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )
  def forward(self, x):
      x = self.layer(x)
      return x

In [13]:
class Attention(nn.Module):
  """
  Question (b)

  Implement Multi-head attention class.
  Please refer to part 3.2 in the paper "Attention is All You Need"
  You can implement without considering heads (i.e. vanilla attention).
  However, your maximum score will be 5 points.

  - Input shape: [batch, number_of_patches, embedding_dimension]
  - Return: [batch, number_of_patches, embedding_dimension]
  """
  def __init__(self, dim, num_heads, dropout = 0.):
    super().__init__()
    """
    Do NOT modify.
    """
    self.head_dim = dim // num_heads
    self.dim = dim
    self.num_heads = num_heads

    self.scale = self.head_dim ** 0.5 # Don't forget scaling!
    # If you are not going to consider head, you need to change self.scale as below.
    # self.scale = self.dim ** 0.5

    self.dropout = nn.Dropout(dropout)

    ##### YOUR CODE #####
    """
    You need to define some layers...
    """
    self.q_linear = nn.Linear(dim, dim)
    self.k_linear = nn.Linear(dim, dim)
    self.v_linear = nn.Linear(dim, dim)
    self.out_linear = nn.Linear(dim, dim)
    #####################

  def forward(self, x):
    ##### YOUR CODE #####
    query = self.q_linear(x)
    key = self.k_linear(x)
    value = self.v_linear(x)

    query = query.reshape(x.shape[0], x.shape[1], self.num_heads, self.head_dim).permute(0, 2, 1, 3)
    key = key.reshape(x.shape[0], x.shape[1], self.num_heads, self.head_dim).permute(0, 2, 1, 3)
    value = value.reshape(x.shape[0], x.shape[1], self.num_heads, self.head_dim).permute(0, 2, 1, 3)

    at_simil = torch.matmul(query, key.transpose(-2, -1)) / self.scale
    at_prob = F.softmax(at_simil, dim=-1)

    out = torch.matmul(at_prob, value)
    out = out.permute(0, 2, 1, 3).reshape(x.shape[0], x.shape[1], x.shape[2])
    out = self.out_linear(out)
    x = out
    #####################
    return x

class Block(nn.Module):
  """
  Question (c)
  Complete Block class.

  - Input shape: [batch, number_of_patches, embedding_dimension]
  - Return: [batch, number_of_patches, embedding_dimension]
  """
  def __init__(self, dim, num_heads, mlp_dim, dropout=0.):
    super().__init__()

    self.LN1 = nn.LayerNorm(dim)
    self.LN2 = nn.LayerNorm(dim)
    ##### YOUR CODE #####
    self.attention = Attention(dim, num_heads, dropout=dropout)
    self.mlp = MLP(dim, mlp_dim, dropout=dropout)
    #####################

  def forward(self, x):
    """
    Hint: Don't forget the residual connections!
    """
    ##### YOUR CODE #####
    x= x + self.attention(self.LN1(x))
    x= x + self.mlp(self.LN2(x))
    #####################
    return x

In [14]:
class ViT(nn.Module):
    def __init__(self, image_shape, patch_size, num_classes, dim, num_heads, depth, mlp_dim, dropout = 0.):
        super().__init__()
        """
        image_shape: [channel, height, width]
        patch_size: [height, width]
        dim: Embedding dimension
        num_heads: Number of heads to be used in Multi-head Attention
        depth: Number of attention blocks to be used
        mlp_dim: Hidden dimension to be used in MLP layer (=feedforward layer)
        """

        image_ch, image_h, image_w = image_shape # image_ch will be 3(RGB 3 channels) for CIFAR10 dataset
        patch_h, patch_w = patch_size

        assert image_h % patch_h == 0 and image_w % patch_w == 0, 'Image height & width must be divisible by those of patch respectively.'
        assert dim % num_heads == 0, 'Embedding dimension should be divisible by number of heads.'
        num_patches = (image_h // patch_h) * (image_w // patch_w) # e.g. [32 x 32] image & [8 x 8] patch size -> [4 x 4 = 16] patches

        ##### YOUR CODE #####
        """Define Patchification using convolution.
        """
        self.patchify = Patchification(image_ch, patch_size, dim)

        """Define Learnable positional encoding, 1+ is for class token.
           Hint: use nn.Parameter
        """
        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))

        """Define Class token which will be prepended to each image.
           Hint: use nn.Parameter
        """
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        #####################


        # Initialize attention blocks
        self.attention_blocks = nn.ModuleList([
            Block(dim, num_heads, mlp_dim, dropout)
            for _ in range(depth)
        ])

        # Classification head, maps the final vector to class dimension.
        self.classification_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )

    def forward(self, img):
        """
        Question (d)

        ViT forward process

        [Hint]
        - After patchification, shape will be [batch, number_of_patches, dim].
        - If you successfully prepend cls_tokens to this batch of patchfied images, shape will be [batch, 1+ number_of_patches, dim].
        - Then simply add the positional embedding.
        - Now the tokens(patches) are ready to go through the attention blocks.
        - After attention operation, classify with class token. (Simply take off it from whole tokens)
        """
        ##### YOUR CODE #####
        #cls_tokens =  # Shape: [batch, 1, dim]
        x = self.patchify(img)
        cls = self.cls_token.expand(x.shape[0], -1, -1)
        x = torch.cat([cls, x], dim=1)
        x = x + self.pos_embedding[:, : (x.shape[1] + 1), :]

        for block in self.attention_blocks:
            x = block(x)

        x = x[:, 0]
        x = self.classification_head(x)

        #####################
        return x

# ViT Image Classification

In [15]:
"""
Make sure your runtime type is GPU and you are using PyTorch version higher than 1.8!

Do NOT modify.
"""
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Using PyTorch version: {}, Device: {}".format(torch.__version__, DEVICE))

Using PyTorch version: 2.5.1+cu124, Device: cuda


In [16]:
"""
You may change some settings including batch size & augmentations.
But if your implementation is correct, default setting is enough to achieve the target performance(i.e. 65%).
"""
BATCH_SIZE = 100

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Prepare Dataset & DataLoader
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [17]:
"""
Question (e)
Train your ViT to achieve 65% of accuarcy.
Feel free to change the hyperparameters.
But again, if your implementation is correct, default setting is enough to achieve the target performance(i.e. 65%).
"""
# Hyperparameters
EPOCHS = 10
patch_size = (4,4)
dim = 128
depth = 8
num_heads = 8
mlp_dim = 256
dropout = 0.2
learning_rate = 0.001

model = ViT(image_shape = (3,32,32), patch_size = patch_size, num_classes = 10, dim = dim, num_heads = num_heads, depth = depth, mlp_dim = mlp_dim, dropout=dropout).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [18]:
"""
Do NOT modify.
It will take less than 8 minutes for training with default setting.
"""
for epoch in range(1, EPOCHS + 1):
    train(model, trainloader, optimizer, criterion, DEVICE)
    test_loss, test_accuracy = evaluate(model, testloader, criterion, DEVICE)
    print("\n[EPOCH: {}], \tModel: ViT, \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]


[EPOCH: 1], 	Model: ViT, 	Test Loss: 0.0134, 	Test Accuracy: 51.92 % 



  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]


[EPOCH: 9], 	Model: ViT, 	Test Loss: 0.0093, 	Test Accuracy: 67.01 % 



  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]


[EPOCH: 10], 	Model: ViT, 	Test Loss: 0.0093, 	Test Accuracy: 66.96 % 

